In [1]:
from pathlib import Path

import geopandas as gpd
import pandas as pd
from pyproj import CRS
from shapely.geometry import Point

from bedrock.gi.ags.read import ags_to_dfs
from bedrock.gi.ags.transform import ags3_db_to_no_gis_brgi_db
from bedrock.gi.concatenate import concatenate_databases
from bedrock.gi.gis_geometry import calculate_gis_geometry
from bedrock.gi.validate import check_brgi_database, check_no_gis_brgi_database
from bedrock.gi.write import write_gi_db_to_excel, write_gi_db_to_gpkg

# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)

In [3]:
cwd = Path.cwd()
gi_dir = cwd.parent / "data" / "ags3" / "kaitak"
gi_files = sorted(list(gi_dir.glob("**/*.ags")))

print(
    f"{len(gi_files)} Ground Investigation files for the area around Kai Tak, Hong Kong."
)
# gi_files

90 Ground Investigation files for the area around Kai Tak, Hong Kong.


In [4]:
crs = CRS(2326)
output_path = cwd / "kaitak_gi"

In [5]:
brgi_db = {}
for gi_file in gi_files:
    print(f"\n🖥️ Processing {gi_file.parent.name}/{gi_file.name} ...")
    with open(gi_file) as ags_file:
        ags3_data = ags_file.read()
    ags3_db = ags_to_dfs(ags3_data)
    report_no = gi_file.parent.name
    ags3_db["PROJ"]["REPORT_NO"] = int(report_no)
    ags3_db["PROJ"]["PROJ_ID"] = (
        report_no + "/" + ags3_db["PROJ"]["PROJ_ID"].astype(str)
    )
    # Remove (Static) CPT AGS 3 group 'STCN' from brgi_db, because CPT data processing needs to be reviewed.
    # Not efficient to create a GIS point for every point where a CPT measures a value.
    if "STCN" in ags3_db.keys():
        del ags3_db["STCN"]

    brgi_db_from_1_ags3_file = ags3_db_to_no_gis_brgi_db(ags3_db, crs)

    print(
        f"🧐 Validating the Bedrock GI database from AGS file {gi_file.parent.name}/{gi_file.name}..."
    )
    check_no_gis_brgi_database(brgi_db_from_1_ags3_file)
    print(
        f"\n✅ Succesfully converted {gi_file.parent.name}/{gi_file.name} to Bedrock GI database and validated!\n"
    )
    print(
        f"🧵 Concatenating Bedrock GI database for {gi_file.parent.name}/{gi_file.name} to existing Bedrock GI database...\n"
    )
    brgi_db = concatenate_databases(brgi_db, brgi_db_from_1_ags3_file)
    # check_no_gis_brgi_database(brgi_db)

# Drop all rows that have completely duplicate rows in the Project table
brgi_db["Project"] = brgi_db["Project"].drop_duplicates()
# Then drop all that unfortunately still have a duplicate project_uid
brgi_db["Project"] = brgi_db["Project"].drop_duplicates(
    subset="project_uid", keep="first"
)

print(
    f"🧐 Checking the concatenated, i.e. combined Bedrock GI database from the {len(gi_files)} provided..."
)
check_no_gis_brgi_database(brgi_db)
print(
    f"\n✅ Succesfully validated that the {len(gi_files)} AGS 3 files provided were put together correctly in a Bedrock GI database."
)



🖥️ Processing 21659/9508008.AGS ...

🚨 CAUTION: The number of columns on line 70 (2) doesn't match the number of columns of group DREM (3)!
DREM headers: ['HOLE_ID', 'DREM_DPTH', 'DREM_REM']
Line 70:      ['"MBH36/01', '57.35   (200 / 70mm)"']

AGS 3 data was read for Project GE/95/08.8
This Ground Investigation data contains groups:
['PROJ', 'HDIA', 'DREM', 'PTIM', 'SAMP', 'FRAC', 'CORE', 'ISPT', 'HOLE', 'IVAN', 'GEOL', 'WETH', 'DETL']

Transforming AGS 3 groups to Bedrock tables...
Transforming AGS 3 group 'PROJ' to Bedrock GI 'Project' table...
Transforming AGS 3 group 'HOLE' to Bedrock GI 'Location' table...
Transforming AGS 3 group 'SAMP' to Bedrock GI 'Sample' table...
Transforming AGS 3 group 'HDIA' to Bedrock GI 'InSitu_HDIA' table...
Transforming AGS 3 group 'DREM' to Bedrock GI 'InSitu_DREM' table...
Transforming AGS 3 group 'PTIM' to Bedrock GI 'InSitu_PTIM' table...
Transforming AGS 3 group 'FRAC' to Bedrock GI 'InSitu_FRAC' table...
Transforming AGS 3 group 'CORE' to Bedr

Projects with lab data:

| REPORT_NO | PROJ_ID  | File name |
|-----------|----------|-----------|
| 21651     | GE/95/04 |9508008.AGS|

In [6]:
brgi_db = calculate_gis_geometry(brgi_db)
check_brgi_database(brgi_db)

Calculating GIS geometry for the Bedrock GI database tables...
Calculating GIS geometry for the Bedrock GI 'Location' table...
Creating 'LonLatHeight' table with GI locations in WGS84 geodetic coordinates...
    WGS84 geodetic coordinates: (Longitude, Latitude, Ground Level Ellipsoidal Height)
Calculating GIS geometry for the Bedrock GI 'Sample' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_HDIA' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_DREM' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_PTIM' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_FRAC' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_CORE' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_ISPT' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_IVAN' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_GEOL' table...
Calculating GIS geometry for the Bedrock GI 'InSitu_WETH' table...
Calculating GIS geometry for the Bedrock

True

In [7]:
# Some ISPT_NVAL (SPT count) are not numeric, e.g. 100/0.29.
# When converting to numeric, these non-numeric values are converted to NaN
brgi_db["InSitu_ISPT"]["ISPT_NVAL"] = pd.to_numeric(
    brgi_db["InSitu_ISPT"]["ISPT_NVAL"], errors="coerce"
)

In [8]:
spt = brgi_db["InSitu_ISPT"].dtypes
spt

HOLE_ID               object
ISPT_TOP             float64
ISPT_SEAT            float64
ISPT_MAIN            float64
ISPT_NPEN            float64
ISPT_NVAL            float64
ISPT_TYPE             object
ISPT_REM              object
ISPT_INC1            float64
ISPT_INC2            float64
ISPT_INC3            float64
ISPT_INC4            float64
ISPT_INC5            float64
ISPT_INC6            float64
ISPT_LAST            float64
project_uid           object
location_uid          object
depth_to_top         float64
ISPT_REP              object
ISPT_CAS             float64
ISPT_WAT             float64
ISPT_PEN1            float64
ISPT_PEN2            float64
ISPT_PEN3            float64
ISPT_PEN4            float64
ISPT_PEN5            float64
ISPT_PEN6            float64
elevation_at_top     float64
geometry            geometry
dtype: object

In [9]:
write_gi_db_to_gpkg(brgi_db, output_path.with_suffix(".gpkg"))


Table names shouldn't contain [':', '/', '\\', '?', '*', '[', ']'] or spaces and shouldn't be longer than 31 characters.
 Replaced '?LEGD' with 'LEGD'.
Ground Investigation data has been written to 'c:\Users\joost\ReposWindows\bedrock-gi\sandbox\kaitak_gi.gpkg'.


In [10]:
write_gi_db_to_excel(brgi_db, output_path.with_suffix(".xlsx"))

Table names shouldn't contain [':', '/', '\\', '?', '*', '[', ']'] or spaces and shouldn't be longer than 31 characters.
 Replaced '?LEGD' with 'LEGD'.
Ground Investigation data has been written to 'c:\Users\joost\ReposWindows\bedrock-gi\sandbox\kaitak_gi.xlsx'.


In [11]:
brgi_db["LonLatHeight"].explore()

In [3]:
kaitak_dir = Path(
    r"C:\Users\joost\GoogleDrive\JoostGevaert_Bedrock\Bedrock\Case Studies\KaiTak\GIS\data"
)
aoi_gdf = gpd.read_file(kaitak_dir / "ShingFungRoad.gpkg", layer="GIAreaOfInterest")
aoi_gdf.to_crs(4326, inplace=True)
gi_lonlat_gdf = gpd.read_file(kaitak_dir / "kaitak_gi.gpkg", layer="LonLatHeight")


In [4]:
gi_in_aoi = gpd.sjoin(
    gi_lonlat_gdf, aoi_gdf.to_crs(gi_lonlat_gdf.crs), predicate="within", how="inner"
)

In [5]:
m = aoi_gdf.explore()
gi_lonlat_gdf.explore(m=m)
gi_in_aoi.explore(m=m, color="red")

# Speckle Stuff

In [ ]:
import datetime as dt

from dotenv import load_dotenv
from specklepy.api import operations
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.objects import Base
from specklepy.objects.geometry import Line, Point
from specklepy.transports.server import ServerTransport


def to_speckle():
    load_dotenv()
    stream_id = "7fbe8ed384"
    hole_table_path = "data/1_split2/HOLE.csv"
    df = pd.read_csv(hole_table_path, index_col=0)
    client, stream_id = get_stream(stream_id)

    # next create a server transport - this is the vehicle through which you will send and receive
    transport = ServerTransport(client=client, stream_id=stream_id)

    hash = create_hash(df, transport)

    commit_hash(hash, client, stream_id)


def get_stream(stream_id):
    # Authenticate with Speckle server
    speckle_server = "app.speckle.systems"
    speckle_token = os.environ["speckle_token"]
    client = SpeckleClient(host=speckle_server)
    account = get_default_account()

    client.authenticate_with_token(speckle_token)

    # create a new stream. this returns the stream id
    if not stream_id:
        stream_id = client.stream.create(name="a shiny new stream")

    # use that stream id to get the stream from the server
    new_stream = client.stream.get(id=stream_id)
    return client, stream_id


def create_hash(df, transport):
    newObj = Base()
    for i, row in df.iterrows():
        x = row["HOLE_NATE"]
        y = row["HOLE_NATN"]
        z_top = row["HOLE_GL"]
        z_bot = z_top - row["HOLE_FDEP"]

        # GisPointElement, GisLineElement lijken niet te werken
        p1 = Point(x=x, y=y, z=z_top)
        p2 = Point(x=x, y=y, z=z_bot)
        line = Line(start=p1, end=p2)
        relevant_cols = [
            "HOLE_ID",
            "HOLE_TYPE",
            "HOLE_STAR",
            "HOLE_LOG",
            "?HOLE_DLOG",
            "?HOLE_CHEK",
            "?HOLE_DCHK",
            "HOLE_REM",
            "?HOLE_FLSH",
            "HOLE_ENDD",
            "HOLE_BACD",
            "HOLE_CREW",
            "HOLE_INCL",
            "HOLE_EXC",
        ]

        for col in relevant_cols:
            line[col] = row[col]

        newObj[f"myline{i}"] = line
        break

    # this serialises the block and sends it to the transport
    hash = operations.send(base=newObj, transports=[transport])
    return hash


def commit_hash(hash, client, stream_id):
    # you can now create a commit on your stream with this object
    commid_id = client.commit.create(
        stream_id=stream_id,
        object_id=hash,
        message=f"these are lines I made in speckle-py at {dt.datetime.now()}",
    )


if __name__ == "__main__":
    to_speckle()
